In [1]:
%load_ext jupyter_cms

In [2]:
import os
from Bio import SeqIO

In [12]:
#here is the in file defined containing alternative 
file_path ='/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v8_1/v8_1_unzip/QC_unpolished/nucmer/nucmer_h_on_pwh_rcoords'
file_name='v8_1_ph_tr_ctg_rn_rcoords_all.fa'
bash_script_q="v8_1_unzip_h_on_pwh_rn_rcoords_all_qmapping.sh"
bash_script_g="v8_1_unzip_h_on_pwh_rn_rcoords_all_gmapping.sh"
outdir = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v8_1/v8_1_unzip/QC_unpolished/nucmer/nucmer_h_on_pwh_rcoords/'

In [13]:
total_seq_recs = 0
p_contigs =[]
a_contigs = []
a_count =0
p_count =0
for seq_record in SeqIO.parse(file_path+'/'+file_name, "fasta"):
    if "_" in list(seq_record.id):
        a_contigs.append(seq_record)
        a_count += 1
    else:
        p_contigs.append(seq_record)
        p_count += 1
    total_seq_recs += 1
total_seq_recs == p_count + a_count

True

In [14]:
os.chdir(outdir)

In [15]:
SeqIO.write(a_contigs,file_name[:-3]+'_a_ctg.fa' , 'fasta')
SeqIO.write(p_contigs,file_name[:-3]+'_p_ctg.fa' , 'fasta')

113

In [7]:
!ls

out.delta                            v4_1_pa_qcc22.pilon_000084F.fa
test                                 v4_1_pa_qcc22.pilon_000085F.fa
v4_1_p000000Fpap.gcoords             v4_1_pa_qcc22.pilon_000086F.fa
v4_1_p000000Fpap_gfiltered.delta     v4_1_pa_qcc22.pilon_000087F.fa
v4_1_p000001Fpap.gcoords             v4_1_pa_qcc22.pilon_000088F.fa
v4_1_p000001Fpap_gfiltered.delta     v4_1_pa_qcc22.pilon_000089F.fa
v4_1_p000002Fpap.gcoords             v4_1_pa_qcc22.pilon_000090F.fa
v4_1_p000002Fpap_gfiltered.delta     v4_1_pa_qcc22.pilon_000091F.fa
v4_1_p000003Fpap.gcoords             v4_1_pa_qcc22.pilon_000092F.fa
v4_1_p000003Fpap_gfiltered.delta     v4_1_pa_qcc22.pilon_000093F.fa
v4_1_p000004Fpap.gcoords             v4_1_pa_qcc22.pilon_000094F.fa
v4_1_p000004Fpap_gfiltered.delta     v4_1_pa_qcc22.pilon_000095F.fa
v4_1_p000005Fpap.gcoords             v4_1_pa_qcc22.pilon_000096F.fa
v4_1_p000005Fpap_gfiltered.delta     v4_1_pa_qcc22.pilon_000096F_a_ctg.fa
v4_1_p000006Fpap.gcoords             v4_1_

In [16]:
outfq = open(bash_script_q, 'w')
outfq.write('#!/bin/bash\n')
outfg = open(bash_script_g, 'w')
outfg.write('#!/bin/bash\n')
for pseq_record in SeqIO.parse(file_name[:-3]+'_p_ctg.fa', 'fasta'):
    p_acontigs = []
    p_contig = pseq_record
    suffix = file_name[:-3]+pseq_record.id.split("|")[0]+"pap"
    p_file = file_name[0:-3]+"_"+pseq_record.id.split("|")[0]+'.fa'
    SeqIO.write(p_contig, p_file, 'fasta')
    for aseq_record in SeqIO.parse(file_name[:-3]+'_a_ctg.fa', 'fasta'):
        if aseq_record.id.split("|")[0].split("_")[0] == pseq_record.id.split("|")[0]:
            p_acontigs.append(aseq_record)
    a_file = file_name[:-3]+"_"+pseq_record.id.split("|")[0]+'_a_ctg.fa'
    #if we have alternative contigs safe those too
    if p_acontigs != []:
        SeqIO.write(p_acontigs, a_file, 'fasta')
        outfq.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfq.write('delta-filter -q '+'out.delta'+" > "+suffix+"_qfiltered.delta\n")
        outfq.write('show-coords -T '+suffix+"_qfiltered.delta > "+suffix+".qcoords\n")
        outfq.write('mummerplot -p '+suffix+'_qfiltered --postscript '+suffix+"_qfiltered.delta\n")
        outfq.write('mummerplot -c -p '+suffix+'_qfiltered_cov --postscript '+suffix+"_qfiltered.delta\n")
        #for g_file bash script
        outfg.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfg.write('delta-filter -g '+'out.delta'+" > "+suffix+"_gfiltered.delta\n")
        outfg.write('show-coords -T '+suffix+"_gfiltered.delta > "+suffix+".gcoords\n")
        outfg.write('mummerplot -p '+suffix+'_gfiltered --postscript '+suffix+"_gfiltered.delta\n")
        outfg.write('mummerplot -c -p  '+suffix+'_gfiltered_cov --postscript '+suffix+"_gfiltered.delta\n")
outfq.close()
outfg.close()